In [3]:
import torch
import torchvision 
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

batch_size = 1000

train_loader = DataLoader(train_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                        batch_size=batch_size,
                        shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
alexnet = models.alexnet(pretrained=True)
model = nn.Sequential(
    torchvision.transforms.Resize((63,63)),
    alexnet,
    nn.ReLU(),
    nn.Linear(1000, 10)
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 1

In [6]:
def run(epochs, optimizer, loss_fn, train_loader, test_loader):
    for epoch in range(epochs):
        for i, (data, labels) in enumerate(train_loader):
            pred = model(data)
            loss = loss_fn(pred, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                end=''
            )

        print('\nTesting the model after epoch:', epoch)
        correct, total = 0, 0
        for _, (data, labels) in enumerate(test_loader):
            pred = model(data)
            for i in range(len(labels)):
                pr = torch.argmax(pred[i], dim=-1)
                if pr == labels[i]:
                    correct += 1
                total += 1
        print(correct, total, correct/total)

In [5]:
run(epochs, optimizer, loss_fn, train_loader, test_loader)

Epoch 1 [50/50] - Loss: 1.9133507013320923
Testing the model after epoch: 0
2486 10000 0.2486


In [7]:
# reset model
model = nn.Sequential(
    torchvision.transforms.Resize((63,63)),
    alexnet,
    nn.ReLU(),
    nn.Linear(1000, 10)
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [8]:
# freeze feature extraction (13 layers)
for i, param in enumerate(model.parameters()):
    if i < 13:
        param.requires_grad = False

In [9]:
run(epochs, optimizer, loss_fn, train_loader, test_loader)

Epoch 1 [50/50] - Loss: 1.4123297929763794
Testing the model after epoch: 0
5026 10000 0.5026


In [10]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 16, 5, 1, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5, 1, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x))) # 28x28 > 14x14
        x = self.pool(self.relu(self.conv2(x))) # 14x14 > 7x7
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x
    
# Create an instance of the model
model = Model()

In [11]:
train_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())

batch_size = 100

train_loader_mnist = DataLoader(train_dataset_mnist,
                        batch_size=batch_size,
                        shuffle=True)

test_loader_mnist = DataLoader(test_dataset_mnist,
                        batch_size=batch_size,
                        shuffle=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
run(epochs, optimizer, loss_fn, train_loader_mnist, test_loader_mnist)

Epoch 1 [600/600] - Loss: 0.037032209336757664
Testing the model after epoch: 0
9781 10000 0.9781


In [13]:
!pip install scipy

In [14]:
test_dataset_svhn = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=ToTensor())

batch_size = 100

test_loader_svhn = DataLoader(test_dataset_svhn,
                        batch_size=batch_size,
                        shuffle=False)

def runtest(test_loader):
    correct, total = 0, 0
    for _, (data, labels) in enumerate(test_loader):
        data = torchvision.transforms.Resize((28,28))(torchvision.transforms.Grayscale()(data))
        pred = model(data)
        for i in range(len(labels)):
            pr = torch.argmax(pred[i], dim=-1)
            if pr == labels[i]:
                correct += 1
            total += 1
    print(correct, total, correct/total)

Using downloaded and verified file: ./data\test_32x32.mat


In [15]:
runtest(test_loader_svhn)

4898 26032 0.18815304240934236
